# Homework 2

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/

Mounted at /content/drive/
/content/drive/My Drive/Colab Notebooks


In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 61.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=95a3cf3890fb86db2629cbc16d1843658e57eeb1c9e4ed701125b8bb06c64082
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html

In [34]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
spark = SparkSession.builder.getOrCreate()

In [27]:
# importing data
products = spark.read.csv('./products.csv', header=True, inferSchema=True)
orders = spark.read.csv('./order_products__train.csv', header=True, inferSchema=True)

In [104]:
# glimpse on products dataframe
products.show()

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
|         6|        Dry Nose Oil|      11|           11|
|         7|Pure Coconut Wate...|      98|            7|
|         8|Cut Russet Potato...|     116|            1|
|         9|Light Strawberry ...|     120|           16|
|        10|Sparkling Orange ...|     115|            7|
|        11|   Peach Mango Juice|      31|            7|
|        12|Chocolate Fudge L...|     119|            1|
|        13|   Saline Nasal Mist|      11|           11|
|        14|Fresh Scent Dishw...|      74|           17|
|        15|Overnight Diapers..

In [29]:
# glimpse on orders dataframe
orders.show()

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
|       1|     13176|                6|        0|
|       1|     47209|                7|        0|
|       1|     22035|                8|        1|
|      36|     39612|                1|        0|
|      36|     19660|                2|        1|
|      36|     49235|                3|        0|
|      36|     43086|                4|        1|
|      36|     46620|                5|        1|
|      36|     34497|                6|        1|
|      36|     48679|                7|        1|
|      36|     46979|                8|        1|
|      38|     11913|                1|        0|


In [30]:
# joining product names for convenience
orders = orders.join(products, on='product_id')
orders.show()

+----------+--------+-----------------+---------+--------------------+--------+-------------+
|product_id|order_id|add_to_cart_order|reordered|        product_name|aisle_id|department_id|
+----------+--------+-----------------+---------+--------------------+--------+-------------+
|     49302|       1|                1|        1|    Bulgarian Yogurt|     120|           16|
|     11109|       1|                2|        1|Organic 4% Milk F...|     108|           16|
|     10246|       1|                3|        0|Organic Celery He...|      83|            4|
|     49683|       1|                4|        0|      Cucumber Kirby|      83|            4|
|     43633|       1|                5|        1|Lightly Smoked Sa...|      95|           15|
|     13176|       1|                6|        0|Bag of Organic Ba...|      24|            4|
|     47209|       1|                7|        0|Organic Hass Avocado|      24|            4|
|     22035|       1|                8|        1|Organic Who

In [35]:
# grouping by baskets
baskets = orders.groupBy('order_id').agg(F.collect_list('product_name').alias('items'))
baskets.show(5, truncate=False)

+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|order_id|items                                                                                                                                                                                                                                                                                                        |
+--------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1       |[Bulgarian Yogurt, Organic 4% Milk Fat Whole Milk C

https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.fpm.FPGrowth.html#pyspark.ml.fpm.FPGrowth

In [45]:
# fitting FPGrowth
fp = FPGrowth(minSupport=0.004, minConfidence=0.05)
fpm = fp.fit(baskets.select(['items']))
fpm.setPredictionCol('newPrediction')

FPGrowthModel: uid=FPGrowth_4571a81d162d, numTrainingRecords=131209

In [51]:
# glimpse on most frequent items
fpm.freqItemsets.orderBy('freq').show(10, truncate=False)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------------------------------------------------------------------+----+
|items                                                              |freq|
+-------------------------------------------------------------------+----+
|[Organic Broccoli, Bag of Organic Bananas]                         |525 |
|[Red Potatoes]                                                     |525 |
|[Broccoli Crown, Large Lemon]                                      |526 |
|[Carrots, Banana]                                                  |527 |
|[Organic Free Range Chicken Broth]                                 |527 |
|[Organic Cilantro, Organic Avocado]                                |528 |
|[Organic Raspberries, Organic Hass Avocado, Bag of Organic Bananas]|531 |
|[Organic Red Onion, Organic Avocado]                               |531 |
|[Total 2% All Natural Low Fat 2% Milkfat Greek Strained Yogurt]    |531 |
|[Michigan Organic Kale, Organic Baby Spinach]                      |532 |
+------------------------

In [96]:
# getting association rules
ar = fpm.associationRules
ar.show(10, truncate=False)

+-----------------------------------+------------------------+-------------------+------------------+---------------------+
|antecedent                         |consequent              |confidence         |lift              |support              |
+-----------------------------------+------------------------+-------------------+------------------+---------------------+
|[Small Hass Avocado]               |[Banana]                |0.27876248791492103|1.953227986587038 |0.006592535573017095 |
|[Apple Honeycrisp Organic]         |[Bag of Organic Bananas]|0.30506216696269983|2.585717174742176 |0.0052359213163731145|
|[Green Bell Pepper]                |[Banana]                |0.24276080920269735|1.7009720717012022|0.004664314185764696 |
|[Boneless Skinless Chicken Breasts]|[Banana]                |0.28065134099616856|1.9664627683844003|0.0044661570471537775|
|[Organic Garnet Sweet Potato (Yam)]|[Bag of Organic Bananas]|0.24610591900311526|2.0860020365942993|0.004816742753926941 |
|[Organi

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [97]:
ar = ar.select(F.lower(F.concat_ws(',', ar.antecedent)).alias('antecedent'), # changing nested lists to single string and also applying lowercasing
               F.lower(F.concat_ws(',', ar.consequent)).alias('consequent'), # changing nested lists to single string and also applying lowercasing
               'confidence',
               'lift',
               'support',
               F.abs(ar.confidence - ar.support).alias('interest_score') # interest score implementation
               )\
       .sort('interest_score', ascending=False) # applying sorting based on interest_score
# displaying dataframe
ar.show(truncate=False)

+------------------------------------------+----------------------+-------------------+------------------+---------------------+-------------------+
|antecedent                                |consequent            |confidence         |lift              |support              |interest_score     |
+------------------------------------------+----------------------+-------------------+------------------+---------------------+-------------------+
|organic raspberries,organic hass avocado  |bag of organic bananas|0.521099116781158  |4.416853618458589 |0.004046978484707604 |0.5170521382964504 |
|organic hass avocado,organic strawberries |bag of organic bananas|0.4613385315139701 |3.910320890272384 |0.005411214169759696 |0.4559273173442104 |
|organic raspberries,organic strawberries  |bag of organic bananas|0.38862275449101796|3.2939795215770014|0.004946307036864849 |0.38367644745415314|
|organic fuji apple                        |banana                |0.37150752225974826|2.60307222515109  |

In [102]:
# filtering rule to exclude any set that contains the word banana
ar.createOrReplaceTempView('df')
query = """
    SELECT * FROM df
    WHERE NOT (antecedent LIKE '%banana%' OR consequent LIKE '%banana%')
"""
result = spark.sql(query)
result.show(truncate=False)

+------------------------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|antecedent                          |consequent          |confidence         |lift              |support              |interest_score     |
+------------------------------------+--------------------+-------------------+------------------+---------------------+-------------------+
|organic raspberries                 |organic strawberries|0.3011179228272629 |3.6267102566772844|0.012727785441547455 |0.28839013738571545|
|bunched cilantro                    |limes               |0.290625           |6.320672240179016 |0.0042527570517266345|0.2863722429482734 |
|jalapeno peppers                    |limes               |0.2880463401790416 |6.264590129048877 |0.0041689213392374   |0.2838774188398042 |
|organic cilantro                    |limes               |0.2855927396483267 |6.2112278761009945|0.007674778406969034 |0.2779179612413577 |
|limes       

In [103]:
# filtering based on lift and support
ar.createOrReplaceTempView('df')
query = """
    SELECT * FROM df
    WHERE lift > 6 AND support > 0.005
"""
spark.sql(query).show(truncate=False)

+--------------------+--------------------+-------------------+------------------+--------------------+-------------------+
|antecedent          |consequent          |confidence         |lift              |support             |interest_score     |
+--------------------+--------------------+-------------------+------------------+--------------------+-------------------+
|organic cilantro    |limes               |0.2855927396483267 |6.2112278761009945|0.007674778406969034|0.2779179612413577 |
|organic garlic      |organic yellow onion|0.20442520442520443|6.252313903829055 |0.006478214146895411|0.197946990278309  |
|organic yellow onion|organic garlic      |0.19813519813519814|6.252313903829056 |0.006478214146895411|0.19165698398830272|
|limes               |organic cilantro    |0.16691529918780043|6.2112278761009945|0.007674778406969034|0.1592405207808314 |
+--------------------+--------------------+-------------------+------------------+--------------------+-------------------+



Above I used the `lift` and `support` metrics to find association rules as they provide strong information. If we look on the findings we can see limes and cilantro are commonly bougth together (vice versa) which makes sense as they are both basic ingredients of Mexican cousine which I love :). The other finding is the garlic and yellow onion pair (vice versa) which are basic ingredients in every cousine.
